<a href="https://colab.research.google.com/github/semwalvikash/AIDEEP/blob/main/Aideep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# 1️⃣ Install required packages
!pip install -q fastapi uvicorn pyngrok transformers accelerate chromadb sentence-transformers pypdf httpx rich


In [28]:
# 2️⃣ Set up directories
!mkdir -p deep-shiva/data/sources

In [30]:
# 3️⃣ models.py
%%writefile deep-shiva/models.py
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

class HFModel:
    def __init__(self, model_name="microsoft/phi-2", device=None):
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if self.device=="cuda" else torch.float32
        ).to(self.device)

    def generate(self, prompt: str, system: str = "", max_new_tokens=300):
        full_prompt = (system + "\n\n" + prompt).strip()
        inputs = self.tokenizer(full_prompt, return_tensors="pt").to(self.device)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)


Writing deep-shiva/models.py


In [31]:
4️⃣ rag.py
%%writefile deep-shiva/rag.py
import chromadb
from sentence_transformers import SentenceTransformer
from pypdf import PdfReader

class RAG:
    def __init__(self):
        self.client = chromadb.PersistentClient(path="rag_store")
        self.collection = self.client.get_or_create_collection("knowledge")
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")

    def ingest_pdf(self, path):
        reader = PdfReader(path)
        chunks = []
        for page in reader.pages:
            text = page.extract_text()
            if text:
                for para in text.split("\n\n"):
                    if len(para.strip()) > 30:
                        chunks.append(para.strip())
        if chunks:
            embeds = self.embedder.encode(chunks).tolist()
            self.collection.add(
                documents=chunks,
                embeddings=embeds,
                metadatas=[{"source": path}] * len(chunks),
                ids=[f"{path}_{i}" for i in range(len(chunks))]
            )

    def search(self, query, k=4):
        emb = self.embedder.encode([query]).tolist()[0]
        results = self.collection.query(query_embeddings=[emb], n_results=k)
        return list(zip(results["documents"][0], results["metadatas"][0]))

    def format_snippets(self, pairs):
        return "\n".join([f"- {doc} (from {meta['source']})" for doc, meta in pairs])


Writing deep-shiva/rag.py


In [32]:
# 5️⃣ prompts.py
%%writefile deep-shiva/prompts.py
SYSTEM_BASE = """You are Deep-Shiva, a healthcare info assistant.
- You are not a doctor.
- Give factual, verified, simple explanations.
- If emergency signs appear, tell user to seek help."""

ANSWER_TEMPLATE = """
User asked: {question}

Relevant knowledge:
{snippets}

Your helpful answer:
"""

Writing deep-shiva/prompts.py


In [33]:
# 6️⃣ safety.py
%%writefile deep-shiva/safety.py
EMERGENCY_KEYWORDS = ["chest pain", "severe bleeding", "suicide", "heart attack"]

EMERGENCY_RESPONSE = (
    "⚠️ This looks like an emergency. Please call your nearest doctor or emergency helpline immediately."
)

def triage_level(query: str):
    q = query.lower()
    if any(word in q for word in EMERGENCY_KEYWORDS):
        return "emergency"
    return "normal"

Writing deep-shiva/safety.py


In [34]:
# 7️⃣ app.py
%%writefile deep-shiva/app.py
from fastapi import FastAPI, UploadFile, Form
from fastapi.responses import JSONResponse
from typing import Optional
from rag import RAG
from prompts import SYSTEM_BASE, ANSWER_TEMPLATE
from safety import triage_level, EMERGENCY_RESPONSE
from models import HFModel

app = FastAPI(title="Deep-Shiva", version="0.1.0")
rag = RAG()
llm = HFModel()

@app.get("/health")
def health():
    return {"status": "ok"}

@app.post("/ingest")
async def ingest(file: UploadFile):
    path = f"deep-shiva/data/sources/{file.filename}"
    with open(path, "wb") as f:
        f.write(await file.read())
    rag.ingest_pdf(path)
    return {"ok": True, "file": file.filename}

@app.post("/chat")
async def chat(query: str = Form(...), session_id: Optional[str] = Form(None)):
    level = triage_level(query)
    if level == "emergency":
        return JSONResponse({"level": level, "answer": EMERGENCY_RESPONSE})

    pairs = rag.search(query, k=4)
    snippets = rag.format_snippets(pairs)
    prompt = ANSWER_TEMPLATE.format(question=query, snippets=snippets)

    answer = llm.generate(prompt, system=SYSTEM_BASE)
    disclaimer = "\n\n— This is general health information, not a diagnosis."

    return {
        "level": level,
        "answer": answer + disclaimer,
        "sources": list({p[1]['source'] for p in pairs}),
        "session": session_id or "anonymous",
    }


Writing deep-shiva/app.py


In [36]:
!ngrok config add-authtoken YOUR_AUTHTOKEN_HERE

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [55]:
!git remote add origin https://github.com/semwalvikash/AIDEEP

fatal: not a git repository (or any of the parent directories): .git
